In [ ]:
# imports
import arviz as az

from matplotlib import pyplot as plt

import pandas as pd

import pymc3 as pm

from sklearn.preprocessing import StandardScaler

from utils.prepare_data import clean_data, filter_rows_by_std, get_magnitude_diffs, load_data

In [ ]:
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

In [ ]:
# load the data
df = load_data()
# load the data
df_mex = load_data(filename='mexico_labeled.csv')

# Data preparation

In [ ]:
df = clean_data(df)  # Basic cleaning
# construct the magnitude differences
ordered_mag_columns = ['magcr3', 'magbr3', 'magar3', 'bpmag', 'gmag', 'rpmag', 'jmag', 'kmag']
df_diffs = get_magnitude_diffs(df, ordered_mag_columns)
# filter Na's and measurements with too large standard deviations
df_diffs_filtered = filter_rows_by_std(df_diffs, df, std_thresholds={
    'sigcr3': 0.05,
    'sigbr3': 0.05,
    'sigar3': 0.05,
    'ejmag': 0.05,
    'ekmag': 0.05,
}).dropna()

In [ ]:
df_mex = clean_data(df_mex)  # Basic cleaning
# construct the magnitude differences
ordered_mag_columns = ['magcr3', 'magbr3', 'magar3', 'bpmag', 'gmag', 'rpmag', 'jmag', 'kmag']
df_diffs_mex = get_magnitude_diffs(df_mex, ordered_mag_columns)
df_diffs_mex['bpmag_rpmag'] = df_mex['bpmag'] - df_mex['rpmag']
# filter Na's and measurements with too large standard deviations
df_diffs_filtered_mex = filter_rows_by_std(df_diffs_mex, df_mex, std_thresholds={
    'sigcr3': 0.05,
    'sigbr3': 0.05,
    'sigar3': 0.05,
    'ejmag': 0.05,
    'ekmag': 0.05,
}).dropna()

df_diffs_filtered_mex = df_diffs_filtered_mex[
    (df_diffs_filtered_mex['magcr3_magbr3']>-50) 
    * (df_diffs_filtered_mex['magcr3_magbr3']<50)
    * (df_diffs_filtered_mex['magbr3_magar3']<50)
]

In [ ]:
# Re-index
df_filter = df.loc[df_diffs_filtered.index]
df_mex_filter = df_mex.loc[df_diffs_filtered_mex.index]

df_mex_filter.index = df_mex_filter.index + 3000
df_diffs_filtered_mex.index = df_diffs_filtered_mex.index + 3000

In [ ]:
# Merge the data
df = pd.concat([df_filter, df_mex_filter])
df_diffs_filtered = pd.concat([df_diffs_filtered, df_diffs_filtered_mex])

# Bayesian Modelling

## Theory

WEB references (all with PyMC3):
- https://docs.pymc.io/en/v3/pymc-examples/examples/generalized_linear_models/GLM-linear.html
- [not-the-most-readable-guide] https://benslack19.github.io/data%20science/statistics/pymc-linreg-entry01/
- [check with hack account] https://towardsdatascience.com/bayesian-linear-regression-in-python-via-pymc3-ab8c2c498211
- https://vincentk1991.github.io/Bayesian-regression-tutorial/ (a study on different connections among predictors and a target variable)
- [with GLM] https://www.quantstart.com/articles/Bayesian-Linear-Regression-Models-with-PyMC3/

## Models depending on field

In [ ]:
X = df_diffs_filtered

giant_indicator = df['logg'][X.index] < 3.5

X = X[giant_indicator]

fields = df['field'][X.index]

In [ ]:
filter_value = df['loggflag'][X.index]>.3*10**8
# vmicroflag : about 3x bigger error on vmicroflag>100 measurements.
# filter_value = df['vmicroflag'][X.index][giant_indicator]>100
print(f'Fall out: {sum(filter_value)}')

In [ ]:
X.shape, filter_value.shape

### Magnitude diff shift from 'star-metric'

In [ ]:
# Predict C-B magnitude diff from temperature, gravity and chemistry and fix intercept on the field

X_use = df[['logg', 'teff', 'cafe', 'feh']].loc[df_diffs_filtered.index]
# replace 'magcr3_magbr3' with 'magbr3_magar3' in the line below
y_use = df_diffs_filtered['magbr3_magar3']
fields_use = df['field'][df_diffs_filtered.index]

giant_indicator = df['logg'][df_diffs_filtered.index] < 3.5
filter_value = df['loggflag'][df_diffs_filtered.index]>.3*10**8

# Filter!
# X_use = X_use[~filter_value]
# y_use = y_use[~filter_value]
# fields_use = fields_use[~filter_value]

ind = X_use.index
cols = X_use.columns
normalizer = StandardScaler()
X_use = normalizer.fit_transform(X_use)
X_use = pd.DataFrame(
    data=X_use,
    index=ind,
    columns=cols
)

In [ ]:
X_use.shape, y_use.shape, fields_use.shape, len(set(fields_use.values))

In [ ]:
with pm.Model() as predictive_model:
    a_teff = pm.Normal('slope_teff', 0, 1)
    a_logg = pm.Normal('slope_logg', 0, 1)
    a_cafe = pm.Normal('slope_cafe', 0, 1)
    a_feh = pm.Normal('slope_feh', 0, 1)
    
    mag_shift = pm.Normal('mag_shift', 0, .1, shape=len(set(fields_use.values)))
    eps = pm.Exponential('error', 1)
    
    # a data container, can be changed
    # observation
    x_teff = pm.Data('x_teff', X_use['teff'].values)
    x_logg = pm.Data('x_logg', X_use['logg'].values)
    x_cafe = pm.Data('x_cafe', X_use['cafe'].values)
    x_feh = pm.Data('x_feh', X_use['feh'].values)
    
    field_index = list(set(fields_use))
    field_index.sort(key=lambda x: int(x.split('-')[1]))
    field_index = {x: e for e, x in enumerate(field_index)}
    field = pm.Data('field', [field_index[f] for f in fields_use.values])
    
    obs = pm.Normal(
        'observation',
        a_teff * x_teff + a_logg * x_logg + a_cafe * x_cafe + a_feh * x_feh + mag_shift[field],
        eps,
        observed=y_use
    )
    
    # Use Maximum A Posteriori (MAP) optimisation as initial value for MCMC
    start = pm.find_MAP()
    # Use the No-U-Turn Sampler
    step = pm.NUTS()

    # use MCMC to sample
    trace = pm.sample(
        draws=8000,
        tune=1000,
        # start=start,
        # 
        return_inferencedata=True)

In [ ]:
trace.posterior.mag_shift.values.mean(axis=1).mean(axis=0)

In [ ]:
mag_shifts = [(f, shift) for f, shift in
              zip(field_index, trace.posterior.mag_shift.values.mean(axis=1).mean(axis=0))]

In [ ]:
az.plot_posterior(trace, var_names=['error', 'mag_shift'], grid=(17, 2))

In [ ]:
mag_shifts

### Diagnostics

In [ ]:
pm.plot_trace(trace)  # Plot trace

In [ ]:
# Check autocorrelation - for all the parameters, for all the chains.
az.plot_autocorr(trace)